In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('/opt/data/fl_voterreg/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian', 'other', 'unknown']) == False].sample(SAMPLE, random_state=21)
del df

# Additional features
sdf['name_last'] = sdf.name_last.str.title()

sdf

,name_last,name_first,race
5577783,Lidros,Johan,nh_white
11025099,Dionne,JOHN,nh_white
9175381,Serrano Ortiz,HILDA,hispanic
511481,Hampton,Tracy,nh_white
336833,Mitchell,MICHAEL,nh_white
...,...,...,...
7765170,Osipov,KONSTANTIN,nh_white
8437763,Ammerman,DANIEL,nh_white
816193,Goldberg,Randolph,nh_white
13135601,Carrick,DONALD,nh_white


In [3]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('./fl_voter_reg/lstm/fl_ln_race.csv', columns=[])
rdf

,name_last
race,
asian,19382
hispanic,167274
nh_black,141448
nh_white,671896


In [4]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,9362
hispanic,41246
nh_black,22837
nh_white,145560


## Preprocessing the input data

In [5]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1151
Max feature len = 29, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [6]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [7]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            36832     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 119,780
Trainable params: 119,780
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
22500/22500 [==============================] - 1420s 61ms/step - loss: 0.6696 - accuracy: 0.7577 - val_loss: 0.6030 - val_accuracy: 0.7804
Epoch 2/15
22500/22500 [==============================] - 1407s 63ms/step - loss: 0.5995 - accuracy: 0.7818 - val_loss: 0.5825 - val_accuracy: 0.7893
Epoch 3/15
22500/22500 [==============================] - 1403s 62ms/step - loss: 0.5808 - accuracy: 0.7882 - val_loss: 0.5717 - val_accuracy: 0.7911
Epoch 4/15
22500/22500 [==============================] - 1410s 63ms/step - loss: 0.5703 - accuracy: 0.7921 - val_loss: 0.5650 - val_accuracy: 0.7944
Epoch 5/15
22500/22500 [==============================] - 1410s 63ms/step - loss: 0.5643 - accuracy: 0.7933 - val_loss: 0.5621 - val_accuracy: 0.7970
Epoch 6/15
22500/22500 [==============================] - 1410s 63ms/step - loss: 0.5595 - accuracy: 0.7956 - val_loss: 0.5581 - val_accuracy: 0.7966
Epoch 7/15
22500/22500 [==============================] - 1404s 62ms/step - loss: 0.5593 - 

In [9]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.5506719946861267
Test accuracy: 0.8000500202178955


## Confusion Matrix

In [10]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

6250/6250 - 67s
              precision    recall  f1-score   support

       asian       0.75      0.38      0.50      3876
    hispanic       0.80      0.81      0.81     33455
    nh_black       0.64      0.21      0.31     28290
    nh_white       0.81      0.93      0.87    134379

    accuracy                           0.80    200000
   macro avg       0.75      0.58      0.62    200000
weighted avg       0.78      0.80      0.77    200000

[[  1475    415     91   1895]
 [    97  27249    237   5872]
 [    83    522   5802  21883]
 [   313   5681   2901 125484]]


## Save model

In [11]:
model.save('./fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [12]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./fl_voter_reg/lstm/fl_all_ln_vocab.csv', index=False, encoding='utf-8')